In [ ]:
!pip install pytelegrambotapi -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.2/232.2 kB 4.3 MB/s eta 0:00:00


In [ ]:
!pip install peft -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 14.2 MB/s eta 0:00:00


In [ ]:
import torch
from peft import PeftModel, PeftConfig, PeftModel, LoraConfig
#from transformers import GenerationConfig
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
import telebot

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
trained_model = '/content/drive/MyDrive/МФТИ/nlp2/chatbot/generative/gpt2results'

config = PeftConfig.from_pretrained(trained_model)

In [ ]:
model_name = 'gpt2-medium'

In [ ]:
# Инициализация токенизатора
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

In [ ]:
trained_model = GPT2LMHeadModel.from_pretrained(
	config.base_model_name_or_path,
	device_map='auto'
	)

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
token = '7160998589:AAFAhNdWC-dWCOXCHnadte5lHvakX-7f5_A'

In [ ]:
# Создаем экземпляр бота
bot = telebot.TeleBot(token)

In [ ]:
def generate_answer(query, model, tokenizer):
    # Токенизация запроса
    encoding = tokenizer(query, return_tensors="pt").to(model.device)

    # Конфигурация генерации текста
    generation_config = {
        "max_length": 250,
        "repetition_penalty": 1.3,
        "pad_token_id": tokenizer.eos_token_id,
        "eos_token_id": tokenizer.eos_token_id
    }

    # Генерация текста ответа
    outputs = model.generate(input_ids=encoding.input_ids, **generation_config, early_stopping=True)

    # Декодирование сгенерированного текста
    text_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return text_output

In [ ]:
# Функция обработки команды /start
@bot.message_handler(commands=["start"])
def start(m):
    bot.send_message(m.chat.id, "Hello, bro!")

# Функция обработки текстовых сообщений
@bot.message_handler(content_types=["text"])
def handle_text(message):
    try:
        # Получение ответа от модели
        answer = generate_answer(message.text, trained_model, tokenizer)

        # Отправка ответа пользователю
        bot.send_message(message.chat.id, answer)
    except Exception as e:
        print("Exception:", e)
        bot.send_message(message.chat.id, "Sorry, I am busy right now.")

In [ ]:
# Запуск бота
bot.polling(none_stop=True, interval=0)